# Song recommender project #

 Garcinuño Feliciano Angela <br>
 Huima Klaara <br>
 Massot Lucas 

In [1]:
import pandas as pd
import numpy as np
import build
import random

In [2]:
data = build.load_data('output.csv')

In [3]:
def recommend(song_list: list, df: pd.core.frame.DataFrame, k: int, distance_matrix = None):
    playlist, indexes = build.format_input(song_list, df)
    if isinstance(distance_matrix, type(None)):
        distances, total_d = build.compute_distance_to_playlist(playlist, indexes, df)
    else:
        distances, total_d = build.select_distance_to_playlist(playlist, indexes, distance_matrix)        
    #display(pd.DataFrame(distances))
    output_df = build.get_k_recommendations(total_d, k, df)
    display(output_df)

In [5]:
# TEST
n = 100
#input = [[str(data.iloc[j]['title']), str(data.iloc[j]['artist_name'])] for j in random.sample(range(len(data)), n)]   # input of song titles
input = [[str(data.iloc[i]['title']), str(data.iloc[i]['artist_name'])] for i in range(n)]
k = 10 #number of song recomendations


# Computing complete distance matrix

#distance_matrix = pd.DataFrame(build.build_distances(data))
#recommend(input, data, k, distance_matrix)


# Computing only distance to playlist
recommend(input, data.copy(), k)

,artist_name,title,release,total distance
9579,Harry Gregson-Williams,Whip Fight,Rundown/Welcome To The Jungle,-340.118617
7126,Nick Ingman,Sangha the Outcast,Two Brothers,-327.752430
7255,Arc Angels,Sent By Angels,Arc Angels,-340.641311
5234,Chorus Line,Mamma pettenami,Coro montenero,-340.239312
7712,Toots Thielemans,Sophisticated Lady,Only Trust Your Heart,-325.450231
431,Paul Rodgers,Alright Now,Alright Now,-327.232613
2008,Van Halen,"Dirty Movies"" (Album Version)",Fair Warning,-327.769679
6108,Shelby Lynne,Old Times Sake,Suit Yourself,-340.911917
4161,Maria Muldaur,It Ain't The Meat It's The Motion (LP Version),Waitress In A Donut Shop,-335.916551
9578,DJ Slugo,Where the Rats 3,Jukin 4 the Haters Vol.2,-325.902601
